In [7]:
import os
from shutil import copyfile
from tqdm.auto import tqdm
import json

def create_yolo_segm_data(json_path,image_path,result_path):
    anns = json.load(open(json_path))
    image_data = {}
    for img_s in anns['images']:
        image_data[img_s['id']] = img_s
        image_data[img_s['id']]['anns'] = []
        
    for ans_s in anns['annotations']:
        image_data[ans_s['image_id']]['anns'].append(ans_s)
    
    for image in tqdm(image_data.values()):
        copyfile(f"{image_path}/{image['file_name']}",f"{result_path}/images/{image['file_name']}")
        label_s = []
        for ann in image['anns']:
            xs = ann['segmentation'][0][::2]
            xs = [x / image['width'] for x in xs]
            ys = ann['segmentation'][0][1::2]
            ys = [y / image['height'] for y in ys]
            m = [ann['category_id']]
            for i in range(len(xs)):
                m.append(xs[i])
                m.append(ys[i])
            m = list(map(str,m))
            label_s.append(' '.join(m) + '\n')
            
        with open(f"{result_path}/labels/{image['file_name'].split('.')[0]}.txt",'w') as f:
            f.writelines(label_s)

In [5]:
!mkdir yolo_data
!mkdir yolo_data/val
!mkdir yolo_data/train

!mkdir yolo_data/val/images
!mkdir yolo_data/val/labels

!mkdir yolo_data/train/images
!mkdir yolo_data/train/labels

mkdir: cannot create directory ‘yolo_data’: File exists
mkdir: cannot create directory ‘yolo_data/val’: File exists
mkdir: cannot create directory ‘yolo_data/train’: File exists
mkdir: cannot create directory ‘yolo_data/val/images’: File exists
mkdir: cannot create directory ‘yolo_data/val/labels’: File exists
mkdir: cannot create directory ‘yolo_data/train/images’: File exists
mkdir: cannot create directory ‘yolo_data/train/labels’: File exists


In [8]:
create_yolo_segm_data('clear_val_annotations.json','./val','./yolo_data/val')

  0%|          | 0/8557 [00:00<?, ?it/s]

In [9]:
create_yolo_segm_data('clear_train_annotations.json','./train','./yolo_data/train')

  0%|          | 0/16631 [00:00<?, ?it/s]